# Heart Attacks are Funny

This is some analysis to save Rich Evans' life

In [ ]:
import pandas as pd

## Read in the data

In [ ]:
heart_df = pd.read_csv('heart.csv')

## Show me that sweet data

In [ ]:
heart_df.head(5)

### Features

1. age - age in years
2. sex - sex (1 = male; 0 = female)
3. cp - chest pain type (1 = typical angina; 2 = atypical angina; 3 = non-anginal pain; 0 = asymptomatic)
4. trestbps - resting blood pressure (in mm Hg on admission to the hospital)
5. chol - serum cholestoral in mg/dl
6. fbs - fasting blood sugar > 120 mg/dl (1 = true; 0 = false)
7. restecg - resting electrocardiographic results (0 = normal; 1 = having ST-T; 2 = hypertrophy)
8. thalach - maximum heart rate achieved
9. exang - exercise induced angina (1 = yes; 0 = no)
10. oldpeak - ST depression induced by exercise relative to rest
11. slope - the slope of the peak exercise ST segment (1 = upsloping; 2 = flat; 3 = downsloping)
12. ca - number of major vessels (0-3) colored by flourosopy
13. thal - 2 = normal; 1 = fixed defect; 3 = reversable defect
14. num - the predicted attribute - diagnosis of heart disease (angiographic disease status) (Value 0 = < diameter narrowing; Value 1 = > 50% diameter narrowing)

Looks like the previous sample had a lot of people who are gonna die. Let's see some live Rich Evans

In [ ]:
good_hearts = heart_df.where(heart_df['output'] == 0).dropna()
good_hearts.head(5)

## Analysis

Let's do some feature selection. Inspired by [this blog](https://towardsdatascience.com/feature-selection-with-pandas-e3690ad8504b)